In [1]:
from pandas import DataFrame as DF
from tkinter import filedialog
from tkinter import Tk
from ciso8601 import parse_datetime as to_date
from time import time

In [2]:
def get_lines():
    lines = []
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    for file in filedialog.askopenfilenames():
        with open(file) as txt:
            for line in txt.readlines():
                yield line

In [10]:
def export_to_csv():
    headers = [
        "ESCSEQ",
        "ENTCODIGO",
        "MRFMESANO",
        "QUAID",
        "TPMOID",
        "CMPID",
        "PLNCODIGO",
        "ESCDATAINICIORO",
        "ESCDATAFIMRO",
        "ESCDATAEMISSRO",
        "ESCVALORMOVRO",
        "ESCDATAINICIORD",
        "ESCDATAFIMRD",
        "ESCDATAEMISSRD",
        "ESCVALORMOVRD",
        "ESCIMPSEG",
        "ESCCODCESS",
        "ESCFREQ",
        "ESCVALORCARO",
        "ESCVALORCARD",
        "ESCVALORCIRO",
        "ESCVALORCIRD",
    ]

    with open("382 planilha.csv", "a+") as csv:
        csv.write(";".join(headers)+"\n")
        for line in get_lines():
            try:
                line = line.strip()
                campos = [
                    line[0:7],
                    line[7:12],
                    to_date(line[12:20]).strftime("%d/%m/%Y"),
                    line[20:23],
                    line[23:27],
                    line[27:31],
                    line[31:37],
                    to_date(line[37:45]).strftime("%d/%m/%Y"),
                    to_date(line[45:53]).strftime("%d/%m/%Y"),
                    to_date(line[53:61]).strftime("%d/%m/%Y"),
                    line[61:74],
                    to_date(line[74:82]).strftime("%d/%m/%Y"),
                    to_date(line[82:90]).strftime("%d/%m/%Y"),
                    to_date(line[90:98]).strftime("%d/%m/%Y"),
                    line[98:111],
                    line[111:127],
                    line[127:132],
                    line[132:136],
                    line[136:149],
                    line[149:162],
                    line[162:175],
                    line[175:188],
                ]
                csv.write(";".join(campos)+"\n")        
            except:
                pass

In [4]:
index = [7, 8, 9, 10]
columns = [str(i) for i in range(1065,1077)]
df_ppng = DF(index=index,columns=columns)
df_ppng.fillna(0.0,inplace=True)

df_rvne= DF(index=index,columns=columns)
df_rvne.fillna(0.0,inplace=True)

df_ppng

,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_calc = to_date(input("Digite a data de cálculo no formato (aaaammdd): "))

for line in get_lines():
    try:
        line = line.strip()
        ESCSEQ = line[0:7]
        ENTCODIGO = line[7:12]
        MRFMESANO = to_date(line[12:20])
        QUAID = line[20:23]
        TPMOID = int(line[23:27])
        CMPID = line[27:31]
        PLNCODIGO = line[31:37]
        ESCDATAINICIORO = to_date(line[37:45])
        ESCDATAFIMRO = to_date(line[45:53])
        ESCDATAEMISSRO = to_date(line[53:61])
        ESCVALORMOVRO = line[61:74]
        ESCDATAINICIORD = to_date(line[74:82])
        ESCDATAFIMRD = to_date(line[82:90])
        ESCDATAEMISSRD = to_date(line[90:98])
        ESCVALORMOVRD = float(line[98:111].replace(",","."))
        ESCIMPSEG = line[111:127]
        ESCCODCESS = line[127:132]
        ESCFREQ = line[132:136]
        ESCVALORCARO = line[136:149]
        ESCVALORCARD = line[149:162]
        ESCVALORCIRO = line[162:175]
        ESCVALORCIRD = line[175:188]

        valor = 0

        if ESCDATAEMISSRD <= data_calc < ESCDATAINICIORD:
                valor = ESCVALORMOVRD
        elif ESCDATAINICIORD <= data_calc < ESCDATAFIMRD:
                valor = ESCVALORMOVRD*((ESCDATAFIMRD-data_calc).days/((ESCDATAFIMRD-ESCDATAINICIORD).days+1))

        df_ppng.loc[TPMOID,CMPID] += valor

        if data_calc < ESCDATAEMISSRD:
            df_rvne.loc[TPMOID,CMPID] += valor    
    except:
        pass

Digite a data de cálculo no formato (aaaammdd): 20190830


In [6]:
ppng = df_ppng.loc[7,"1065"] + df_ppng.loc[7,"1066"] - df_ppng.loc[7,"1067"]
ppng += df_ppng.loc[8,"1065"] + df_ppng.loc[8,"1066"] - df_ppng.loc[8,"1067"]
ppng += -df_ppng.loc[9,"1065"] - df_ppng.loc[9,"1066"] + df_ppng.loc[9,"1067"]
ppng += -df_ppng.loc[10,"1065"] - df_ppng.loc[10,"1066"] + df_ppng.loc[10,"1067"]
ppng

7490407.960312499

In [8]:
ppng_rvne = df_rvne.loc[7,"1065"] + df_rvne.loc[7,"1066"] - df_rvne.loc[7,"1067"]
ppng_rvne += df_rvne.loc[8,"1065"] + df_rvne.loc[8,"1066"] - df_rvne.loc[8,"1067"]
ppng_rvne += -df_rvne.loc[9,"1065"] - df_rvne.loc[9,"1066"] + df_rvne.loc[9,"1067"]
ppng_rvne += -df_rvne.loc[10,"1065"] - df_rvne.loc[10,"1066"] + df_rvne.loc[10,"1067"]
ppng_rvne

7480053.791249998

In [12]:
export_to_csv()